In [2]:
import os
import json

In [3]:
source = "offline_cache/"
target = "offline_cache_cleaned/"

In [11]:
for cache in os.listdir("offline_cache/"):
    filename = (source + cache)
    max_bus = int(cache.split("_")[-3])//3+1
    with open(filename) as f:
        res = {}
        data = json.load(f)
        for hour in range(0,48,2):
            res2={}
            for hub in ['JFK','LGA','PENN']:
                schedule = json.loads(data[str(hour)][hub])
                bus_route_str = []
                bus_route = []
                bus_num = []
                bus_gap = []
                bus_routename = []
                # MSG_TYPE, Bus_currenthour
                total_bus = 0
                for i in range(len(schedule['Bus_route'])):
                    if str(schedule['Bus_route'][i]) not in bus_route_str:
                        if(schedule['Bus_num'][i] > 0):
                            bus_route_str.append(str(schedule['Bus_route'][i]))
                            bus_route.append(schedule['Bus_route'][i])
                            bus_num.append(schedule['Bus_num'][i])
                            total_bus += schedule['Bus_num'][i]
                            bus_gap.append(schedule['Bus_gap'][i])
                            bus_routename.append(schedule['Bus_routename'][i])
                    else:
                        j = bus_route_str.index(str(schedule['Bus_route'][i]))
                        bus_gap[j] = (bus_gap[j]*bus_num[j]+schedule['Bus_gap'][i]*schedule['Bus_num'][i])/(schedule['Bus_num'][i]+bus_num[j])
                        bus_num[j] += schedule['Bus_num'][i]
                        total_bus += schedule['Bus_num'][i]
                schedule={"Bus_route":bus_route, "Bus_num":bus_num,\
                         "Bus_gap":bus_gap,"Bus_routename":bus_routename,\
                         "MSG_TYPE": "BUS_SCHEDULE",\
                         "Bus_currenthour":str(hour)}
                # Post process, in case the optimization violating its constraints..
                if(total_bus > max_bus):
                    discount_rate = max_bus/total_bus
                    for i in range(len(schedule['Bus_route'])):
                        schedule['Bus_num'][i] = round(schedule['Bus_num'][i] * discount_rate)
                        schedule['Bus_gap'][i] = schedule['Bus_gap'][i] / discount_rate
                res2[hub] = json.dumps(schedule)
            res[str(hour)] = res2
    filename = (target + cache)
    with open(filename,'w') as f:
        json.dump(res,f)